In [ ]:
from csv import reader
from numpy import array, vstack, size, float_, copy
from os import listdir, path
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
def retrieve_DataInstance(
    param_CSV
):
    CSVReader = reader(param_CSV, delimiter = ',')
    Data = list(CSVReader)
    YInstance = array(Data.pop(0))
    YInstance = YInstance.astype(float_)
    XInstance = array(Data).reshape(-1)
    XInstance = XInstance.astype(float_)
    
    return XInstance, YInstance

In [ ]:
X = []
Y = []
_index = 1
for DataFile in listdir("../public/data/processed_data"):
    CSVFile = open(path.join("../public/data/processed_data", DataFile), 'r')
    XInstance, YInstance = retrieve_DataInstance(CSVFile)
    CSVFile.close()
    X.append(XInstance)       
    Y.append(YInstance)
    # if _index == 100:
    #     break
    _index += 1
X = vstack(X)
Y = vstack(Y)
    
XTrainValidate, XTest, YTrainValidate, YTest = train_test_split(X, Y, test_size = 0.2)
    
# 10-Fold PCA cross-validating
Size = 0
ReducedSizeSet = [*range(1, int(0.8*_index))]
MeanAccuracy = 0
for ReducedSize in ReducedSizeSet: 
    _XTrainValidate = copy(XTrainValidate)
    PCATransformer = PCA(n_components = ReducedSize)
    PCATransformer.fit(_XTrainValidate)
    _XTrainValidate = PCATransformer.transform(_XTrainValidate)
    Accuracies = []
    KFoldSampler = KFold(n_splits = 10, shuffle = True, random_state = 1801)
    for _index_Train, _index_Validate in KFoldSampler.split(_XTrainValidate, YTrainValidate):
        XTrain = XTrainValidate[_index_Train]
        YTrain = YTrainValidate[_index_Train]
        XValidate = XTrainValidate[_index_Validate]
        YValidate = YTrainValidate[_index_Validate]
        Model = LinearRegression()
        Model.fit(XTrain, YTrain)
        YPredict = Model.predict(XValidate)
        Accuracy = mean_squared_error(YValidate, YPredict)
        Accuracies.append(Accuracy)
    _MeanAccuracy = sum(Accuracies) / len(Accuracies)
    print(f"{MeanAccuracy} - {ReducedSize}")
    if(_MeanAccuracy > MeanAccuracy):
        Size = ReducedSize
        MeanAccuracy = _MeanAccuracy

# Linear Regression
PCATransformer = PCA(n_components = Size)
PCATransformer.fit(XTrainValidate)
XTrainValidate = PCATransformer.transform(XTrainValidate)
Model = LinearRegression()
Model.fit(XTrainValidate, YTrainValidate)
YPredict = Model.predict(XTest)